# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
!jupyter nbextension enable --py gmaps


# Import API key
from api_keys import g_key

# Configure gmaps with API key
gmaps.configure(api_key=g_key)

Enabling notebook extension jupyter-gmaps/extension...
      - Validating: OK


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_data_df = pd.read_csv('output_data/weather_data_df.csv')

weather_data_df.head()

,City,Cloudiness,Country,Date,Humidity,Max_Temp,Lat,Lng,Wind_Speed
0,Ponta Delgada,75,PT,1588039151,93,62.600,37.73,-25.67,6.20
1,Stony Plain,75,CA,1588039151,37,53.600,53.53,-114.00,3.60
2,Rikitea,25,PF,1588039151,82,77.144,-23.12,-134.97,6.03
3,Talnakh,25,RU,1588039151,85,21.200,69.49,88.40,4.00
4,Grindavik,75,IS,1588039152,75,41.000,63.84,-22.43,0.50


In [3]:
weather_data_df.count()

City          537
Cloudiness    537
Country       534
Date          537
Humidity      537
Max_Temp      537
Lat           537
Lng           537
Wind_Speed    537
dtype: int64

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
# HINT: be sure to handle NaN values
weather_data_df = weather_data_df.dropna()

lat_lng = weather_data_df[['Lat', 'Lng']].astype(float)

# Fill NaN values and convert to float
humid = weather_data_df["Humidity"]


In [80]:
# Plot Heatmap
fig = gmaps.figure(center=(0, 0), zoom_level=2.5)

# Plot Heatmap
heat_layer = gmaps.heatmap_layer(lat_lng, weights=humid, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)


# Add heat layer
fig.add_layer(heat_layer)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))

In [81]:
data = weather_data_df[(weather_data_df["Max_Temp"] > 0)]
data.head()

,City,Cloudiness,Country,Date,Humidity,Max_Temp,Lat,Lng,Wind_Speed
0,Ponta Delgada,75,PT,1588039151,93,62.600,37.73,-25.67,6.20
1,Stony Plain,75,CA,1588039151,37,53.600,53.53,-114.00,3.60
2,Rikitea,25,PF,1588039151,82,77.144,-23.12,-134.97,6.03
3,Talnakh,25,RU,1588039151,85,21.200,69.49,88.40,4.00
4,Grindavik,75,IS,1588039152,75,41.000,63.84,-22.43,0.50


### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [82]:
# Narrow down the DataFrame to find ideal weather condition for vacation
expected_temperature = (weather_data_df.Max_Temp > 68) & (weather_data_df.Max_Temp < 80)
expected_windSpeed = weather_data_df.Wind_Speed < 11
expected_cloudiness = weather_data_df.Cloudiness == 0
ideal_weather = expected_temperature & expected_windSpeed & expected_cloudiness 

# Use boolean indexing to filter the weather_df dataframe
ideal_weather_df = weather_data_df[ideal_weather]
ideal_weather_df = ideal_weather_df.dropna()
ideal_weather_df = ideal_weather_df.reset_index()
ideal_weather_df.head()


,index,City,Cloudiness,Country,Date,Humidity,Max_Temp,Lat,Lng,Wind_Speed
0,12,Xiazhen,0,CN,1588039153,28,68.270,34.80,117.11,4.78
1,37,Arraial do Cabo,0,BR,1588039156,69,74.174,-22.97,-42.03,1.97
2,40,Hengshui,0,CN,1588038950,28,74.732,37.73,115.70,7.02
3,68,Jardim,0,BR,1588039161,57,73.544,-21.48,-56.14,3.47
4,83,Gwadar,0,PK,1588039164,87,79.052,25.12,62.33,2.21


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [83]:
hotel_df = ideal_weather_df

# Add column for Hotel Name - Note that we used "" to specify initial entry.
hotel_df['Hotel Name'] = ""

# hotel_df.drop(columns=["Hotel_Name"])
hotel_df.head(20)

,index,City,Cloudiness,Country,Date,Humidity,Max_Temp,Lat,Lng,Wind_Speed,Hotel Name
0,12,Xiazhen,0,CN,1588039153,28,68.270,34.80,117.11,4.78,
1,37,Arraial do Cabo,0,BR,1588039156,69,74.174,-22.97,-42.03,1.97,
2,40,Hengshui,0,CN,1588038950,28,74.732,37.73,115.70,7.02,
3,68,Jardim,0,BR,1588039161,57,73.544,-21.48,-56.14,3.47,
4,83,Gwadar,0,PK,1588039164,87,79.052,25.12,62.33,2.21,
5,96,Umluj,0,SA,1588039165,50,74.012,25.02,37.27,0.52,
6,139,Bajos de Haina,0,DO,1588039172,75,76.550,18.42,-70.03,1.10,
7,145,Poum,0,NC,1588039173,68,78.638,-20.23,164.02,6.41,
8,163,Burla,0,IN,1588039175,57,79.340,21.50,83.87,1.19,
9,170,Manama,0,BH,1588039176,50,77.000,26.22,50.58,3.10,


In [84]:
# find the closest hotel 

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "lodging",
    "keyword": "Hotel",
    "key": g_key
}

# Use the lat/lng to identify hotels
for index, row in hotel_df.iterrows():
    # get lat, lng from hotel_df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # assemble url and make API request
    # print(f"Retrieving Results for Index {index}: {restr_type}.")
    response = requests.get(base_url, params=params).json()

    # extract results
    results = response['results']
#     print(results)
    try:
            print(f"Closest hotel is {results[0]['name']}.")

            hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
                  
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")


Closest hotel is Heyun Holiday Inn.
------------
Closest hotel is Hotel da Canoa.
------------
Closest hotel is Sunshine Hotel.
------------
Closest hotel is Hotel Jardim.
------------
Closest hotel is Sadaf Resort.
------------
Closest hotel is عروس الشاطئ للوحدات السكنية المفروشة.
------------
Missing field/result... skipping.
------------
Missing field/result... skipping.
------------
Missing field/result... skipping.
------------
Closest hotel is Four Seasons Hotel Bahrain Bay.
------------
Closest hotel is Sentidos Beach Retreat.
------------
Closest hotel is Pousada Portobello.
------------
Closest hotel is Radisson Blu Hotel, Dakar Sea Plaza.
------------
Closest hotel is Guzhen Hotel.
------------
Missing field/result... skipping.
------------
Missing field/result... skipping.
------------
Missing field/result... skipping.
------------
Closest hotel is Pakistan Hotel Dalbandin.
------------
Closest hotel is Kampô SPA HOTEL.
------------
Closest hotel is Hotel Posada Navideña.
-

In [87]:
hotel_df

,index,City,Cloudiness,Country,Date,Humidity,Max_Temp,Lat,Lng,Wind_Speed,Hotel Name
0,12,Xiazhen,0,CN,1588039153,28,68.270,34.80,117.11,4.78,Heyun Holiday Inn
1,37,Arraial do Cabo,0,BR,1588039156,69,74.174,-22.97,-42.03,1.97,Hotel da Canoa
2,40,Hengshui,0,CN,1588038950,28,74.732,37.73,115.70,7.02,Sunshine Hotel
3,68,Jardim,0,BR,1588039161,57,73.544,-21.48,-56.14,3.47,Hotel Jardim
4,83,Gwadar,0,PK,1588039164,87,79.052,25.12,62.33,2.21,Sadaf Resort
5,96,Umluj,0,SA,1588039165,50,74.012,25.02,37.27,0.52,عروس الشاطئ للوحدات السكنية المفروشة
6,139,Bajos de Haina,0,DO,1588039172,75,76.550,18.42,-70.03,1.10,
7,145,Poum,0,NC,1588039173,68,78.638,-20.23,164.02,6.41,
8,163,Burla,0,IN,1588039175,57,79.340,21.50,83.87,1.19,
9,170,Manama,0,BH,1588039176,50,77.000,26.22,50.58,3.10,Four Seasons Hotel Bahrain Bay


In [88]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]
marker_locations = locations


In [89]:
# Convert hotel name to a list
hotel_name = hotel_df["Hotel Name"].tolist()

In [90]:
# Create a marker_layer using the hotel name list to fill the info box
markers = gmaps.marker_layer(marker_locations,
    info_box_content=[f"Hotel: {name}" for name in hotel_name])

# Add marker layer
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))